In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator, train_discriminator, train_generator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [2]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="1"
print(tf.test.is_gpu_available())

True


In [3]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [4]:
## reduce redundant features which can be assembled
dataset=df.drop(['NIHTotal','THD_ID','cortical_CT', 'subcortical_CT',
              'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
              'CT_left', 'CT_right',],axis=1)

In [5]:
sc = MinMaxScaler()
dataset.loc[:,dataset.columns] = sc.fit_transform(dataset.loc[:,dataset.columns])


In [6]:
## setting hyperparameter
latent_dim = dataset.shape[1]
epochs = 40000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [7]:
generator = Generator(latent_dim)
discriminator = Discriminator()

In [8]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [9]:
gen_opt = tf.keras.optimizers.Adam(0.00001, 0.5)
disc_opt = tf.keras.optimizers.Adam(0.00001,)

In [10]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [11]:
X_train, X_test = train_test_split(dataset, test_size=0.2,shuffle=True,
                                   stratify=dataset['elapsed_class'],
                                   random_state=None)

In [ ]:
# separate to 0,1 dataset
data_1=X_train.loc[X_train['elapsed_class']==1]
data_0=X_train.loc[X_train['elapsed_class']==0]
## store losses
### generator losses
losses_gen = np.array([])
best_loss_gen = np.inf
### discriminator losses
losses_dis = np.array([])
best_loss_dis = np.inf

for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0

    # resample the dataset
    data1_shape_0 = data_1.sample(data_0.shape[0])
    df_same_shape = pd.concat([data1_shape_0, data_0]).to_numpy()

#     # slices to data and labels
#     df_same_ = df_same_shape.iloc[:, :-1].to_numpy()
#     org_label = df_same_shape.iloc[:, -1].to_numpy().reshape(-1, 1)

    # create batch dataset
    training_dataset = tf.data.Dataset.from_tensor_slices(df_same_shape)\
        .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

    for data in training_dataset:
        for _ in range(n_critic): # 5*discriminator times, 1*generator times
            disc_loss += train_discriminator(data, generator,
                                             discriminator, disc_opt, latent_dim)
#         if disc_opt.iterations.numpy() % n_critic == 0: ### using samples
        gen_loss+= train_generator(data, generator,
                                    discriminator, gen_opt, batch_size, latent_dim)
    
    losses_gen= np.append(losses_gen, gen_loss / batch_size)
    losses_dis= np.append(losses_dis, disc_loss / (batch_size*n_critic))
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1, time.time() - start,
                                                                               gen_loss / batch_size,
                                                                               disc_loss / (batch_size*n_critic)))
    # save best discriminator or generator
    if abs(best_loss_gen) > abs((gen_loss / batch_size)):
        best_loss_gen = (gen_loss / batch_size)
        generator.save_weights(checkpoint_prefix+"gen", save_format='tf')
        
    if abs(best_loss_dis) > abs((disc_loss / (batch_size*n_critic))):
        best_loss_dis = (disc_loss / (batch_size*n_critic))
        discriminator.save_weights(checkpoint_prefix+"dis", save_format='tf')
    


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1 is 13.511372089385986 sec - gen_loss = 82.87185704629562, disc_loss = 57.554768394044004
Time for epoch 2 is 3.3122072219848633 sec - gen_loss = 81.43757749654343, disc_loss = 35.54769649383508
Time for epoch 3 is 4.196262359619141 sec - gen_loss = 80.31027989406145, disc_loss = 22.625542549842322
Time for epoch 4 is 7.79648756980896 sec - gen_loss = 80.45792498607076, disc_loss = 14.772373782561553
Time for epoch 5 is 2.5681605339050293 sec - gen_loss = 78.46915134692071, disc_loss = 9

Time for epoch 71 is 2.08413028717041 sec - gen_loss = 70.29751282341445, disc_loss = -0.027299607368065616
Time for epoch 72 is 2.3041441440582275 sec - gen_loss = 68.82424655052526, disc_loss = -0.02812214731554502
Time for epoch 73 is 2.5961620807647705 sec - gen_loss = 69.7313588915647, disc_loss = -0.028304376060708597
Time for epoch 74 is -11.932487964630127 sec - gen_loss = 67.34351995880684, disc_loss = -0.028528451465391208
Time for epoch 75 is 3.372210741043091 sec - gen_loss = 66.78754352860102, disc_loss = -0.02875991203986512
Time for epoch 76 is 4.63228964805603 sec - gen_loss = 66.295347567366, disc_loss = -0.029062817117449413
Time for epoch 77 is 9.244577884674072 sec - gen_loss = 67.40315423888063, disc_loss = -0.029026668817245815
Time for epoch 78 is 0.8960559368133545 sec - gen_loss = 67.96190252511205, disc_loss = -0.029727008288221828
Time for epoch 79 is 0.9280576705932617 sec - gen_loss = 67.48924122530153, disc_loss = -0.030397468052916328
Time for epoch 80 is

Time for epoch 146 is 2.140133857727051 sec - gen_loss = 45.60214156919289, disc_loss = -0.02904764433316695
Time for epoch 147 is 2.068129301071167 sec - gen_loss = 45.41603817433604, disc_loss = -0.02893578359584032
Time for epoch 148 is 2.448152780532837 sec - gen_loss = 43.81393719987709, disc_loss = -0.02876487694865847
Time for epoch 149 is -13.377272844314575 sec - gen_loss = 45.0649507658732, disc_loss = -0.02866120839600672
Time for epoch 150 is 4.172260761260986 sec - gen_loss = 44.27328981345531, disc_loss = -0.02850647675759671
Time for epoch 151 is 4.648290395736694 sec - gen_loss = 43.89039399758545, disc_loss = -0.028337803939387368
Time for epoch 152 is 5.584348917007446 sec - gen_loss = 44.64848932942611, disc_loss = -0.02812202769719435
Time for epoch 153 is 0.8920557498931885 sec - gen_loss = 43.56402162117006, disc_loss = -0.028068293751951134
Time for epoch 154 is 0.8880555629730225 sec - gen_loss = 43.83187327625418, disc_loss = -0.027842198631028336
Time for epoc

Time for epoch 221 is 2.3921494483947754 sec - gen_loss = 35.21101743872219, disc_loss = -0.021808270234584853
Time for epoch 222 is 2.5161571502685547 sec - gen_loss = 36.20281641229902, disc_loss = -0.021783704811461038
Time for epoch 223 is 2.4801549911499023 sec - gen_loss = 35.30051496446625, disc_loss = -0.02185239838703118
Time for epoch 224 is 3.1361961364746094 sec - gen_loss = 35.15139799522412, disc_loss = -0.021726468055148537
Time for epoch 225 is -15.040013551712036 sec - gen_loss = 36.786453664619806, disc_loss = -0.021622273266619134
Time for epoch 226 is 7.264454126358032 sec - gen_loss = 35.65192699675186, disc_loss = -0.021534472851853597
Time for epoch 227 is 3.2482030391693115 sec - gen_loss = 36.141443653008544, disc_loss = -0.021476530617228456
Time for epoch 228 is 0.9440591335296631 sec - gen_loss = 35.70131930652697, disc_loss = -0.02144354438954403
Time for epoch 229 is 0.9160571098327637 sec - gen_loss = 36.195696618275534, disc_loss = -0.02139815400105411
T

Time for epoch 296 is 2.62416410446167 sec - gen_loss = 33.058954619391166, disc_loss = -0.019278292249095726
Time for epoch 297 is 2.916182279586792 sec - gen_loss = 32.5736904523344, disc_loss = -0.018946146735350618
Time for epoch 298 is 3.1241953372955322 sec - gen_loss = 33.01789292872579, disc_loss = -0.019122798337418805
Time for epoch 299 is 4.160260200500488 sec - gen_loss = 31.949017602113376, disc_loss = -0.019015343978477994
Time for epoch 300 is -14.27003264427185 sec - gen_loss = 31.75236448452115, disc_loss = -0.018915197197235466
Time for epoch 301 is 3.476217269897461 sec - gen_loss = 32.45264235762466, disc_loss = -0.019140696070362614
Time for epoch 302 is 0.9280579090118408 sec - gen_loss = 32.704855647272936, disc_loss = -0.019119503578158885
Time for epoch 303 is 0.9320583343505859 sec - gen_loss = 32.69275079120512, disc_loss = -0.018944191663490047
Time for epoch 304 is 0.9280579090118408 sec - gen_loss = 33.12255748860746, disc_loss = -0.01897533499014265
Time 

...

Time for epoch 4335 is 1.2360773086547852 sec - gen_loss = 14.23465257158598, disc_loss = -0.005713396597713667
Time for epoch 4336 is 1.3080816268920898 sec - gen_loss = 14.372237998675988, disc_loss = -0.00569475620331354
Time for epoch 4337 is 1.2960810661315918 sec - gen_loss = 14.405158115779752, disc_loss = -0.005707896830653036
Time for epoch 4338 is 1.2760798931121826 sec - gen_loss = 14.126958848728554, disc_loss = -0.0058162305133536854
Time for epoch 4339 is 1.3800864219665527 sec - gen_loss = 14.498085415692477, disc_loss = -0.005674791335409997
Time for epoch 4340 is 1.4760921001434326 sec - gen_loss = 14.631205535430517, disc_loss = -0.005683161427412607
Time for epoch 4341 is 1.500093698501587 sec - gen_loss = 14.488076354051342, disc_loss = -0.00547100918313215
Time for epoch 4342 is 1.5560972690582275 sec - gen_loss = 14.573356567908055, disc_loss = -0.0057927911455134765
Time for epoch 4343 is 1.6481032371520996 sec - gen_loss = 14.195481905568428, disc_loss = -0.0056

Time for epoch 4408 is 0.7960498332977295 sec - gen_loss = 14.286691904459515, disc_loss = -0.00570076313627888
Time for epoch 4409 is 0.9520595073699951 sec - gen_loss = 14.087037309027043, disc_loss = -0.005588861109924953
Time for epoch 4410 is 1.0760669708251953 sec - gen_loss = 14.038981585720853, disc_loss = -0.005771352212306302
Time for epoch 4411 is 1.0960686206817627 sec - gen_loss = 14.356624314186291, disc_loss = -0.005762330387642457
Time for epoch 4412 is 1.0600664615631104 sec - gen_loss = 14.201695135611134, disc_loss = -0.005359856158905308
Time for epoch 4413 is 1.120069980621338 sec - gen_loss = 14.231942552276323, disc_loss = -0.00557893852517505
Time for epoch 4414 is 0.9560596942901611 sec - gen_loss = 13.795061280442402, disc_loss = -0.005353603364616813
Time for epoch 4415 is 0.9840614795684814 sec - gen_loss = 13.99166498771217, disc_loss = -0.005503267285571503
Time for epoch 4416 is 1.124070167541504 sec - gen_loss = 14.240639307429118, disc_loss = -0.0057172

Time for epoch 4481 is 1.0080630779266357 sec - gen_loss = 14.284243759180802, disc_loss = -0.005769137637630178
Time for epoch 4482 is 1.0160634517669678 sec - gen_loss = 13.974659869088976, disc_loss = -0.005410527962242518
Time for epoch 4483 is 1.044065237045288 sec - gen_loss = 13.863856200436599, disc_loss = -0.005617959547120256
Time for epoch 4484 is 1.048065423965454 sec - gen_loss = 13.787354994346025, disc_loss = -0.005568054157023124
Time for epoch 4485 is 1.0480656623840332 sec - gen_loss = 14.026537075985035, disc_loss = -0.005705599387624913
Time for epoch 4486 is 1.0960683822631836 sec - gen_loss = 14.018565128495872, disc_loss = -0.005393461443442304
Time for epoch 4487 is 1.1080694198608398 sec - gen_loss = 13.69982502833254, disc_loss = -0.005362723231944148
Time for epoch 4488 is 1.1440715789794922 sec - gen_loss = 14.017423209194131, disc_loss = -0.005471420165770639
Time for epoch 4489 is 1.144071340560913 sec - gen_loss = 13.975015316634373, disc_loss = -0.005657

Time for epoch 4554 is -20.343077421188354 sec - gen_loss = 13.811043983530828, disc_loss = -0.005421191147438345
Time for epoch 4555 is 1.024064064025879 sec - gen_loss = 13.613443310655349, disc_loss = -0.005494609047552054
Time for epoch 4556 is 1.0040628910064697 sec - gen_loss = 13.537090991421248, disc_loss = -0.005302469808936105
Time for epoch 4557 is 1.0400645732879639 sec - gen_loss = 13.493376932195755, disc_loss = -0.005213255607499355
Time for epoch 4558 is 1.0920681953430176 sec - gen_loss = 13.40939650239897, disc_loss = -0.005586622121884357
Time for epoch 4559 is 1.0840678215026855 sec - gen_loss = 13.67951160186486, disc_loss = -0.005520956620246373
Time for epoch 4560 is 1.1120693683624268 sec - gen_loss = 13.569072086961302, disc_loss = -0.005674229198299191
Time for epoch 4561 is 1.1320710182189941 sec - gen_loss = 13.485699789736294, disc_loss = -0.005509665736079505
Time for epoch 4562 is 1.1640727519989014 sec - gen_loss = 13.952389785830732, disc_loss = -0.0055

Time for epoch 4627 is 0.9800610542297363 sec - gen_loss = 13.581303911925064, disc_loss = -0.0055521970269810945
Time for epoch 4628 is 0.9920620918273926 sec - gen_loss = 13.153243873129357, disc_loss = -0.005292248476998414
Time for epoch 4629 is 1.0200636386871338 sec - gen_loss = 13.43241245435228, disc_loss = -0.005275906869413118
Time for epoch 4630 is -19.100645065307617 sec - gen_loss = 13.971677956828714, disc_loss = -0.005303271039475716
Time for epoch 4631 is 1.0680668354034424 sec - gen_loss = 13.441028500703565, disc_loss = -0.005294759556019912
Time for epoch 4632 is 1.0600664615631104 sec - gen_loss = 13.404290339512741, disc_loss = -0.005340296465673496
Time for epoch 4633 is 1.0920684337615967 sec - gen_loss = 13.2924461687783, disc_loss = -0.00523860865581239
Time for epoch 4634 is 1.144071340560913 sec - gen_loss = 13.243238534786686, disc_loss = -0.005386059240506209
Time for epoch 4635 is 1.1520719528198242 sec - gen_loss = 13.30280078255171, disc_loss = -0.005272

## show the training results

In [ ]:
plt.title("ADS-GAN training Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.grid()
plt.plot(losses_gen, label='Generator')
plt.plot(losses_dis, label='Discriminator')
plt.legend()
plt.show()

In [ ]:
## create matrix 0 row*latent_dim columns
arr=np.empty((0,latent_dim))
noise = tf.random.normal([128, latent_dim])

## batch testing data
testing_dataset = tf.data.Dataset.from_tensor_slices(X_test.to_numpy())\
        .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## generate dataset
for data in testing_dataset:
    gen_=generator(noise, data).numpy()
    arr=np.append(arr,gen_,axis=0)
arr.shape

In [ ]:
output_dataset = pd.DataFrame(np.round(sc.inverse_transform(arr)), columns=[
    'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
    'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
    'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
    'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
    'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
    'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
    'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
    'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
    'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
    'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
    'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
    'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
    'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
    'NIHS_10_in', 'NIHS_11_in','elapsed_class'
])
output_dataset

In [ ]:
# output_dataset.to_csv("../dataset/output_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')

In [ ]:
output_dataset = pd.DataFrame(np.round(sc.inverse_transform(X_test)), columns=[
    'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
    'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
    'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
    'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
    'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
    'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
    'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
    'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
    'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
    'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
    'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
    'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
    'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
    'NIHS_10_in', 'NIHS_11_in','elapsed_class'
])
output_dataset